In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset

In [2]:
#Load the dataset using dataset using pandas
data = pd.read_csv('diabetes.csv')

In [3]:
data

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure,Triceps skin fold thickness,2-Hour serum insulin,Body mass index,Age,Class
0,6,148,72,35,0,33.6,50,positive
1,1,85,66,29,0,26.6,31,negative
2,8,183,64,0,0,23.3,32,positive
3,1,89,66,23,94,28.1,21,negative
4,0,137,40,35,168,43.1,33,positive
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,63,negative
764,2,122,70,27,0,36.8,27,negative
765,5,121,72,23,112,26.2,30,negative
766,1,126,60,0,0,30.1,47,positive


In [4]:
# For x: Extract out the dataset from all the rows (all samples) and all columns except last column (all features). (input)
# For y: Extract out the last column (which is the label) (output)
# Convert both to numpy using the .values method
#iloc is used to take a piece of the csv => syntax [start_row : end_row, start_column : end_coulmn]
x = data.iloc[:, 0:-1].values
y_string = list(data.iloc[:, -1])

In [5]:
x

array([[  6. , 148. ,  72. , ...,   0. ,  33.6,  50. ],
       [  1. ,  85. ,  66. , ...,   0. ,  26.6,  31. ],
       [  8. , 183. ,  64. , ...,   0. ,  23.3,  32. ],
       ...,
       [  5. , 121. ,  72. , ..., 112. ,  26.2,  30. ],
       [  1. , 126. ,  60. , ...,   0. ,  30.1,  47. ],
       [  1. ,  93. ,  70. , ...,   0. ,  30.4,  23. ]])

In [6]:
y_string

['positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',

In [7]:
#Encoding 'positive' and 'negative'
y_int = []
for s in y_string:
    if s== 'positive':
        y_int.append(1)
    else:
        y_int.append(0)

In [8]:
#Now converting it into an array
y = np.array(y_int, dtype='float64')
y

array([1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1.,
       1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0.,
       0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.,
       1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1.,
       1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1.,
       1., 0., 0., 0., 0.

We can perform normalization using this
### $x^{\prime}=\frac{x-\mu}{\sigma}$

In [9]:
#Feature normalization. All features should have the same range of values (-1, 1)
sc = StandardScaler()
x = sc.fit_transform(x)
x

array([[ 0.63994726,  0.84832379,  0.14964075, ..., -0.69289057,
         0.20401277,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.69289057,
        -0.68442195, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -0.69289057,
        -1.10325546, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ...,  0.27959377,
        -0.73518964, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.69289057,
        -0.24020459,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.69289057,
        -0.20212881, -0.87137393]])

In [10]:
#Converting arrays into pyTorch tensors
x = torch.tensor(x)
y = torch.tensor(y).unsqueeze(1)
print(x.shape)
print(y.shape)

torch.Size([768, 7])
torch.Size([768, 1])


In [11]:
y

tensor([[1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
      

In [12]:
class Dataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return len(self.x)
    

dataset = Dataset(x, y)

In [13]:
len(dataset)

768

In [14]:
#Load the dataset to your datalaoder for batch processing and shuffling
train_loader = torch.utils.data.DataLoader(dataset=dataset, 
                            batch_size=5, 
                            shuffle=True)

train_loader

In [15]:
#Have a look at the data loader
print("There is {} batches in the dataset".format(len(train_loader)))
for (x,y) in train_loader:
    print("For one iteration (batch), there is:")
    print("Data:    {}".format(x.shape))
    print("Labels:  {}".format(y.shape))
    break

There is 154 batches in the dataset
For one iteration (batch), there is:
Data:    torch.Size([5, 7])
Labels:  torch.Size([5, 1])


![demo](https://user-images.githubusercontent.com/30661597/60379583-246e5e80-9a68-11e9-8b7f-a4294234c201.png)

In [16]:
#Building the above neural network
class Model(nn.Module):
    def __init__(self, input_features, output_features):
        super(Model, self).__init__() #Inheriting from the nn.Module
        self.fc1 = nn.Linear(input_features, 5) #syntax: nn.Linear(no_of_input_features, no_of_output_features)
        self.fc2 = nn.Linear(5, 4)
        self.fc3 = nn.Linear(4, 3)
        self.fc4 = nn.Linear(3, output_features)
        self.sigmoid = nn.Sigmoid()    
        self.tanh = nn.Tanh()

    #Tanh activation function between the layers and sigmoid at the end
    def forward(self, x):
        out = self.fc1(x)
        out = self.tanh(out)
        out = self.fc2(out)
        out = self.tanh(out)
        out = self.fc3(out)
        out = self.tanh(out) 
        out = self.fc4(out)
        out = self.sigmoid(out)
        return out

$H_{p}(q)=-\frac{1}{N} \sum_{i=1}^{N} y_{i} \cdot \log \left(p\left(y_{i}\right)\right)+\left(1-y_{i}\right) \cdot \log \left(1-p\left(y_{i}\right)\right)$


cost = -(Y * torch.log(hypothesis) + (1 - Y) * torch.log(1 - hypothesis)).mean()

In [17]:
#Create the network (an object of the Net class)
net = Model(x.shape[1], 1)
#In Binary Cross Entropy: the input and output should have the same shape 
#size_average = True --> the losses are averaged over observations for each minibatch
criterion = torch.nn.BCELoss(size_average=True)
# We will use SGD with momentum with a learning rate of 0.1
#Syntax torch.optim.SGD(weights, learning_rate, momentum)
optimizer = torch.optim.SGD(net.parameters(), 
                            lr=0.1, 
                            momentum=0.9)

c:\Users\varni\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [19]:
# Train the network 
epochs = 200

#Train the network
for epoch in range(epochs):
    for inputs, labels in train_loader:
        inputs = inputs.float()
        labels = labels.float()

        # Forward propagation
        output = net(inputs)
        # Loss calculation
        loss = criterion(output, labels)
        # Clear the gradient buffer (w = w - lr*gradient)
        optimizer.zero_grad()
        # Backward propagation
        loss.backward()
        # Weight optimization
        optimizer.step()

    #Accuracy
    #Since we are using a sigmoid, we will need to perform some thresholding
    output = (output>0.5).float()
    # Accuracy: (output == labels).float().sum() / output.shape[0]
    accuracy = (output == labels).float().mean()
    # Print statistics 
    print("Epoch {}/{}, Loss: {:.3f}, Accuracy: {:.3f}".format(epoch+1,epochs, loss, accuracy))



Epoch 1/200, Loss: 0.529, Accuracy: 0.667
Epoch 2/200, Loss: 0.422, Accuracy: 1.000
Epoch 3/200, Loss: 0.592, Accuracy: 0.667
Epoch 4/200, Loss: 0.706, Accuracy: 0.667
Epoch 5/200, Loss: 0.171, Accuracy: 1.000
Epoch 6/200, Loss: 0.852, Accuracy: 0.667
Epoch 7/200, Loss: 0.827, Accuracy: 0.333
Epoch 8/200, Loss: 0.647, Accuracy: 0.667
Epoch 9/200, Loss: 0.253, Accuracy: 1.000
Epoch 10/200, Loss: 0.832, Accuracy: 0.333
Epoch 11/200, Loss: 0.865, Accuracy: 0.333
Epoch 12/200, Loss: 0.467, Accuracy: 1.000
Epoch 13/200, Loss: 0.024, Accuracy: 1.000
Epoch 14/200, Loss: 0.849, Accuracy: 0.333
Epoch 15/200, Loss: 1.381, Accuracy: 0.333
Epoch 16/200, Loss: 0.609, Accuracy: 1.000
Epoch 17/200, Loss: 0.793, Accuracy: 0.333
Epoch 18/200, Loss: 1.155, Accuracy: 0.333
Epoch 19/200, Loss: 0.488, Accuracy: 0.667
Epoch 20/200, Loss: 0.064, Accuracy: 1.000
Epoch 21/200, Loss: 0.054, Accuracy: 1.000
Epoch 22/200, Loss: 0.935, Accuracy: 0.333
Epoch 23/200, Loss: 0.191, Accuracy: 1.000
Epoch 24/200, Loss: 